In [1]:
import numpy as np
from src.utils import *
from tqdm import tqdm

unary_operators=[np.sin, np.cos, np.exp, np.abs, np.log, np.tan]
binary_operators=[np.add, np.subtract, np.multiply, np.divide]


operators = unary_operators + binary_operators

In [2]:
def evolve(x,y):
    try:
        NUM_POPULATION = 100
        NUM_GENERATIONS = 600
        MAX_DEPTH_INITIAL = 5
        DEDUPE_INTERVAL = 15
        ELITISM = False
        ELITE_COUNT = 3
        TOURNAMENT_SUBSET_SIZE = 3
        TOURNAMENT_WINNER_SIZE = 20
        XOVER = 0.8
        MUTATION = 0.8
        BREED_NEW = 15
        KILL_AGE = 16
        CONSTANT_FIT_INTERVAL = 30
        CONSTANT_FIT_ITERATION = 6

        print("Population is creating with a size: ", NUM_POPULATION)
        my_population = create_population(NUM_POPULATION,MAX_DEPTH_INITIAL,x.shape[0])
        print("Population has created.")
        print("Individual fitness values are assigning")
        assign_population_fitness(my_population,x,y)
        print(f" Population size: {len(my_population)}")
        print(f"Population mean fitness: {calculate_mean_fitness(my_population)}")
        print(f"Population mean complexity: {calculate_mean_complexity(my_population)}")

        # Sort the population by fitness (ascending), then take the top 5
        top_5_individuals = top_n_individuals(my_population, 5)

        # Print the top 5 individuals with the minimum fitness
        for i, individual in enumerate(top_5_individuals, 1):
            print(f"Top {i}: Fitness = {individual.fitness}: Genome = {individual.genome}")
        #----------------------------------------------------------------------------
        assign_population_fitness(my_population,x,y)
        best_result = top_n_individuals(my_population, 1)[0]
        for generation in tqdm(range(NUM_GENERATIONS)):
            num_xover=0
            num_mut=0
            num_succesfull_mut=0
            age_population(my_population)
            

            # Kill eldest individuals
            kill_eldest(my_population, KILL_AGE)
            # print("Population size before killing constants: ", len(my_population))
            
            # print("Population size after killing constants: ", len(my_population))


            best_ind = tournament_selection(my_population,TOURNAMENT_SUBSET_SIZE,TOURNAMENT_WINNER_SIZE,ELITISM=ELITISM, elite_count=ELITE_COUNT)

            
            for _ in range(BREED_NEW): # CHECK
                if random.random() < XOVER:
                    # Xover the best individual and replace worst individuals
                    parent1 = my_population[best_ind[np.random.choice(len(best_ind))]]
                    parent2 = my_population[best_ind[np.random.choice(len(best_ind))]]
                    child1, child2 = crossover(parent1,parent2)
                    my_population.append(child1)
                    my_population.append(child2)
                    num_xover+=1

                if random.random() < MUTATION:
                    # Mutate best individuals and replace worst individuals
                    parent = my_population[best_ind[np.random.choice(len(best_ind))]]
                    
                    child, success = mutation_w_sa(parent, x.shape[0], x, y)
                    num_mut+=1
                    if success:                
                        my_population.append(child)
                        num_succesfull_mut+=1

            assign_population_fitness(my_population,x,y)
            # print(f"xover: {num_xover}, mut: {num_mut}, succesfull mut: {num_succesfull_mut}")
            simplify_constant_population(my_population)
            # Update elites age as 0
            elites = top_n_individuals(my_population, ELITE_COUNT)
            if best_result.fitness > elites[0].fitness:
                best_result = elites[0]
                print(f"New best result found: {best_result}")
            
            for elite in elites:
                my_population.remove(elite)
                elite.age = 0
                my_population.append(elite)

            # Deduplicate every few iterations
            if generation % DEDUPE_INTERVAL == DEDUPE_INTERVAL-1:
                init_population = len(my_population)
                my_population = deduplicate_population(my_population)
                dedup_population = len(my_population)
                simplify_operation_population(my_population)
                simplified_operations = len(my_population)
                kill_constant(my_population)
                no_constant = len(my_population)

                print(f"Initial: {init_population}, Deduplicated: {init_population-dedup_population}, Operation simplified: {dedup_population-simplified_operations}, Constant: {simplified_operations-no_constant}, Final: {no_constant}")
                print(f"Population mean complexity: {calculate_mean_complexity(my_population)}")
                print(f"Population mean fitness: {calculate_mean_fitness(my_population)}")
                print()
                best_5 = top_n_individuals(my_population, 5)
                for ind in range(len(best_5)):
                    print(f"{ind}. Fitness: {best_5[ind].fitness:.3f}, Genome: {best_5[ind].genome}, Age: {best_5[ind].age}")
            #     # Fit constants of every individual
            if generation % CONSTANT_FIT_INTERVAL == CONSTANT_FIT_INTERVAL-1:
                print(f"Generation {generation}: Mean fitness {calculate_mean_fitness(my_population)}")
                for ind in range(len(my_population)):
                    stronger_inv = fit_constants(my_population[ind],CONSTANT_FIT_ITERATION, x, y)
                    my_population[ind] = stronger_inv
                print(f"Population mean fitness after constant fit: {calculate_mean_fitness(my_population)}")
                best_5 = top_n_individuals(my_population, 5)
                for ind in range(len(best_5)):
                    print(f"{ind}. Fitness: {best_5[ind].fitness:.3f}, Genome: {best_5[ind].genome}, Age: {best_5[ind].age}")
            

        my_population = deduplicate_population(my_population)
        #-------------------------------------------------------------------------------------------------------
        assign_population_fitness(my_population,x,y)
        # Sort the population by fitness (ascending), then take the top 5
        top_5_individuals = top_n_individuals(my_population, 15)

        # Print the best individual
        print(f"Best individual: Fitness = {best_result.fitness}: Genome = {best_result.genome}")
        # Print the top 5 individuals with the minimum fitness in population
        for i, individual in enumerate(top_5_individuals, 1):
            print(f"Top {i}: Fitness = {individual.fitness}: Genome = {individual.genome}")

        # Print the mean fitness of the population
        print("Mean fitness of the population: ", calculate_mean_fitness(my_population))
        print("Population size: ", len(my_population))
        print(f"Population mean complexity: {calculate_mean_complexity(my_population)}")
        
        return my_population
    except:
        print("ERROR")
        return my_population

In [3]:
try:
    problem = np.load("data/problem_0.npz")
    x = problem["x"]
    y = problem["y"]
    print("x.shape:", x.shape)
    print("y.shape:", y.shape)
    my_pop_0 = evolve(x,y)
except:
    print("ERROR")

x.shape: (2, 1000)
y.shape: (1000,)
Population is creating with a size:  100
Population has created.
Individual fitness values are assigning
 Population size: 69
Population mean fitness: 3.7094940078423864e+103
Population mean complexity: 246.17391304347825
Top 1: Fitness = 0.21878415413758193: Genome = (x[0] + x[1])
Top 2: Fitness = 2.7836504775468094: Genome = abs(sin((tan(exp(x[0])) - sin(exp(x[0])))))
Top 3: Fitness = 3.3938689706093967: Genome = ([0.19655428] * [0.26409258])
Top 4: Fitness = 3.3944628114054063: Genome = cos([-1.52966511])
Top 5: Fitness = 3.3954610008132042: Genome = abs([-0.11965962])


  0%|          | 1/600 [00:02<20:32,  2.06s/it]

New best result found: Individual(genome=x[0], fitness=np.float64(0.01067566400550203), age=0, T=1)


  2%|▎         | 15/600 [00:17<07:21,  1.32it/s]

Initial: 581, Deduplicated: 359, Operation simplified: 0, Constant: 114, Final: 108
Population mean complexity: 265.6111111111111
Population mean fitness: 2.3699545050104135e+103

0. Fitness: 0.011, Genome: x[0], Age: 0
1. Fitness: 0.219, Genome: (x[1] + x[0]), Age: 3
2. Fitness: 0.328, Genome: (x[0] - cos(abs(exp(x[1])))), Age: 9
3. Fitness: 1.136, Genome: (x[0] * abs(x[1])), Age: 11
4. Fitness: 1.994, Genome: sin(sin(x[0])), Age: 13


...

Initial: 511, Deduplicated: 312, Operation simplified: 0, Constant: 67, Final: 132
Population mean complexity: 13.469696969696969
Population mean fitness: 4237953.976721494

0. Fitness: 0.011, Genome: x[0], Age: 0
1. Fitness: 0.139, Genome: (x[0] - sin(((x[1] - sin(abs(x[1]))) * x[1]))), Age: 7
2. Fitness: 0.141, Genome: (x[0] + sin(sin(x[1]))), Age: 1
3. Fitness: 0.145, Genome: (x[0] - sin((((x[1] - sin(abs(x[1]))) * (x[0] - sin(x[1]))) * (x[1] * x[1])))), Age: 14
4. Fitness: 0.160, Genome: (x[0] + sin(sin((x[1] - sin(abs(sin(x[1]))))))), Age: 10


 30%|██▉       | 179/600 [05:41<03:51,  1.82it/s]

Initial: 540, Deduplicated: 354, Operation simplified: 0, Constant: 59, Final: 127
Population mean complexity: 22.62992125984252
Population mean fitness: 4404782.399455512

0. Fitness: 0.011, Genome: x[0], Age: 0
1. Fitness: 0.120, Genome: (x[0] + sin(sin(sin(x[1])))), Age: 11
2. Fitness: 0.130, Genome: (x[0] - sin((abs(sin((x[1] * x[1]))) / x[0]))), Age: 7
3. Fitness: 0.139, Genome: (x[0] - sin(((x[1] - sin(abs(x[1]))) * x[1]))), Age: 13
4. Fitness: 0.141, Genome: (x[0] + sin(sin(x[1]))), Age: 16
Generation 179: Mean fitness 4404782.399455512


 30%|██▉       | 179/600 [05:48<13:39,  1.95s/it]

ERROR


In [ ]:
try:
    problem = np.load("data/problem_1.npz")
    x = problem["x"]
    y = problem["y"]
    print("x.shape:", x.shape)
    print("y.shape:", y.shape)
    my_pop_1 = evolve(x,y)
except:
    print("ERROR")

In [ ]:
try:
    problem = np.load("data/problem_2.npz")
    x = problem["x"]
    y = problem["y"]
    print("x.shape:", x.shape)
    print("y.shape:", y.shape)
    my_pop_2 = evolve(x,y)
except:
    print("ERROR")

In [ ]:
try:
    problem = np.load("data/problem_3.npz")
    x = problem["x"]
    y = problem["y"]
    print("x.shape:", x.shape)
    print("y.shape:", y.shape)
    my_pop_3 = evolve(x,y)
except: 
    print("ERROR")

In [ ]:
try:
    problem = np.load("data/problem_4.npz")
    x = problem["x"]
    y = problem["y"]
    print("x.shape:", x.shape)
    print("y.shape:", y.shape)
    my_pop_4 = evolve(x,y)
except: 
    print("ERROR")

In [ ]:
try:
    problem = np.load("data/problem_5.npz")
    x = problem["x"]
    y = problem["y"]
    print("x.shape:", x.shape)
    print("y.shape:", y.shape)
    my_pop_5 = evolve(x,y)
except: 
    print("ERROR")

In [ ]:
try:
    problem = np.load("data/problem_6.npz")
    x = problem["x"]
    y = problem["y"]
    print("x.shape:", x.shape)
    print("y.shape:", y.shape)
    my_pop_6 = evolve(x,y)
except: 
    print("ERROR")

In [ ]:
try:
    problem = np.load("data/problem_7.npz")
    x = problem["x"]
    y = problem["y"]
    print("x.shape:", x.shape)
    print("y.shape:", y.shape)
    my_pop_7 = evolve(x,y)
except: 
    print("ERROR")

In [ ]:
try:
    problem = np.load("data/problem_8.npz")
    x = problem["x"]
    y = problem["y"]
    print("x.shape:", x.shape)
    print("y.shape:", y.shape)
    my_pop_8 = evolve(x,y)
except: 
    print("ERROR")